## This program creates a DataFrame of all batting data, categorizes, and adds to separate sheets.

#### Importing packages

In [75]:
import pybaseball
import os
import openpyxl
import statsapi
import pandas as pd
import numpy as np
import time
from pybaseball import cache
cache.enable()

##### Setting Working Directory

In [76]:
# Get the current working directory  
current_dir = os.getcwd() 
program_dir = r'C:\Users\jvgas\OneDrive\Desktop\learning\github\baseballStatistics\py4da\baseballStatistics\programs\programs'
os.chdir(program_dir)

 
print("Current Working Directory:", program_dir)  
time.sleep(3)

data_dir = r'C:\Users\jvgas\OneDrive\Desktop\learning\github\baseballStatistics\py4da\baseballStatistics\datasets\jimbo datasets'
print('')
print("Current Saving Directory:", data_dir)

Current Working Directory: C:\Users\jvgas\OneDrive\Desktop\learning\github\baseballStatistics\py4da\baseballStatistics\programs\programs

Current Saving Directory: C:\Users\jvgas\OneDrive\Desktop\learning\github\baseballStatistics\py4da\baseballStatistics\datasets\jimbo datasets


### Creating battingData DataFrame

In [ ]:
from pybaseball import batting_stats
battingData = batting_stats(2025,2025, qual=1)

HTTPError: Error accessing 'https://www.fangraphs.com/leaders-legacy.aspx'. Received status code 500

In [ ]:
res = battingData.columns
print(res)

print(battingData)

Index(['IDfg', 'Season', 'Name', 'Team', 'Age', 'G', 'AB', 'PA', 'H', '1B',
       ...
       'maxEV', 'HardHit', 'HardHit%', 'Events', 'CStr%', 'CSW%', 'xBA',
       'xSLG', 'xwOBA', 'L-WAR'],
      dtype='object', length=320)
      IDfg  Season              Name   Team  Age    G   AB   PA    H   1B  \
2    15640    2025       Aaron Judge    NYY   33  152  541  679  179   94   
13   21534    2025       Cal Raleigh    SEA   28  159  596  705  147   63   
53   25764    2025    Bobby Witt Jr.    KCR   25  157  623  687  184  108   
6    19755    2025     Shohei Ohtani    LAD   30  158  611  727  172   83   
26   22799    2025   Geraldo Perdomo    ARI   25  161  597  720  173  115   
..     ...     ...               ...    ...  ...  ...  ...  ...  ...  ...   
542  19734    2025        Josh Rojas    CHW   31   69  189  211   34   23   
611  13723    2025   Jacob Stallings  - - -   35   43  119  129   16   12   
534  35041    2025    Jac Caglianone    KCR   22   62  210  232   33   19   
53

### Creating different dictionaries for subsects of stats

In [ ]:
standardStats = ['IDfg', 'Season', 'Name', 'Team', 'Age', 'G', 'AB', 'PA', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'AVG']
fantasyStats = ['IDfg', 'Season', 'Name', 'Team', 'Age', 'R', 'HR', '1B', '2B', '3B', 'H', 'BB', 'IBB', 'SO', 'SB', 'RBI']
advancedStats = ['IDfg', 'Season', 'Name', 'Team', 'Age', 'BB%', 'K%', 'BB/K', 'AVG', 'OBP', 'SLG', 'OPS', 'ISO', 'Spd', 'BABIP','UBR', 'wGDP', 'wSB', 'wRC','wRAA', 'wOBA','wRC+']
statcastStats = ['IDfg', 'Season', 'Name', 'Team', 'Age', 'Events', 'EV', 'maxEV', 'LA', 'Barrels', 'Barrel%', 'HardHit', 'HardHit%', 'AVG', 'xBA', 'SLG', 'xSLG', 'wOBA', 'xwOBA']
battedBallStats = ['IDfg', 'Season', 'Name', 'Team', 'Age','GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%', 'BUH%','Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%']
moreBattedBallStats = ['IDfg', 'Season', 'Name', 'Team', 'Age','GB', 'FB', 'LD', 'IFFB', 'IFH', 'BU', 'BUH', 'Balls', 'Strikes', 'Pitches']
winProbabilityStats = ['IDfg', 'Season', 'Name', 'Team', 'Age','WPA', '-WPA', '+WPA', 'RE24', 'REW', 'pLI', 'phLI', 'PH', 'WPA/LI', 'Clutch']
pitchTypeVeloStats = ['IDfg', 'Season', 'Name', 'Team', 'Age', 'FA% (sc)', 'FT% (sc)', 'FC% (sc)', 'FS% (sc)', 'FO% (sc)', 'SI% (sc)', 'SL% (sc)', 'CU% (sc)', 'KC% (sc)', 'EP% (sc)', 'CH% (sc)', 'SC% (sc)', 'KN% (sc)', 'UN% (sc)', 'vFA (sc)', 'vFT (sc)', 'vFC (sc)', 'vFS (sc)', 'vFO (sc)', 'vSI (sc)', 'vSL (sc)', 'vCU (sc)', 'vKC (sc)', 'vEP (sc)', 'vCH (sc)', 'vSC (sc)', 'vKN (sc)','FA-X (sc)', 'FT-X (sc)', 'FC-X (sc)', 'FS-X (sc)', 'FO-X (sc)', 'SI-X (sc)', 'SL-X (sc)', 'CU-X (sc)', 'KC-X (sc)', 'EP-X (sc)', 'CH-X (sc)', 'SC-X (sc)', 'KN-X (sc)', 'FA-Z (sc)', 'FT-Z (sc)', 'FC-Z (sc)', 'FS-Z (sc)', 'FO-Z (sc)', 'SI-Z (sc)', 'SL-Z (sc)', 'CU-Z (sc)', 'KC-Z (sc)', 'EP-Z (sc)', 'CH-Z (sc)', 'SC-Z (sc)', 'KN-Z (sc)']
pitchValueStats = ['IDfg', 'Season', 'Name', 'Team', 'Age','wFA (sc)', 'wFT (sc)', 'wFC (sc)', 'wFS (sc)', 'wFO (sc)', 'wSI (sc)', 'wSL (sc)', 'wCU (sc)', 'wKC (sc)', 'wEP (sc)', 'wCH (sc)', 'wSC (sc)', 'wKN (sc)', 'wFA/C (sc)', 'wFT/C (sc)', 'wFC/C (sc)', 'wFS/C (sc)', 'wFO/C (sc)', 'wSI/C (sc)', 'wSL/C (sc)', 'wCU/C (sc)', 'wKC/C (sc)', 'wEP/C (sc)', 'wCH/C (sc)', 'wSC/C (sc)', 'wKN/C (sc)']
plateDisciplineStats = ['IDfg', 'Season', 'Name', 'Team', 'Age',  'O-Swing% (sc)', 'Z-Swing% (sc)', 'Swing% (sc)', 'O-Contact% (sc)', 'Z-Contact% (sc)', 'Contact% (sc)', 'Zone% (sc)', 'F-Strike%', 'SwStr%', 'CStr%', 'CSW%', 'Pace']

print('Dictionaries of Stats to be Extracted Created Successfully.')

Dictionaries of Stats to be Extracted Created Successfully.


### Creating differnt DataFrames for subsects of stats

In [ ]:
dfStandardStats = battingData[standardStats]
dfFantasyStats = battingData[fantasyStats]
dfAdvancedStats = battingData[advancedStats]
dfStatcastStats = battingData[statcastStats]
dfBattedBallStats = battingData[battedBallStats]
dfMoreBattedBallStats = battingData[moreBattedBallStats]
dfWinProbabilityStats = battingData[winProbabilityStats]
dfPitchTypeVeloStats = battingData[pitchTypeVeloStats]
dfPitchValueStats = battingData[pitchValueStats]
dfPlateDisciplineStats = battingData[plateDisciplineStats]

### Creating an excel file with different sheets of subsect of stats

In [ ]:
# Saving battingData to xlsx

# Change CWD
os.chdir(data_dir) 

with pd.ExcelWriter('FanGraphs Batting Data April 2025.xlsx',) as writer:  

    dfStandardStats.to_excel(writer, sheet_name='standardStats')
    dfFantasyStats.to_excel(writer, sheet_name = 'fantasyStats')
    dfAdvancedStats.to_excel(writer, sheet_name='advancedStats')
    dfStatcastStats.to_excel(writer, sheet_name='statcastStats')
    dfBattedBallStats.to_excel(writer, sheet_name='battedBallStats')
    dfMoreBattedBallStats.to_excel(writer, sheet_name='moreBattedBallStats')
    dfWinProbabilityStats.to_excel(writer, sheet_name='winProbabilityStats')
    dfPitchTypeVeloStats.to_excel(writer, sheet_name='pitchTypeVeloStats')
    dfPitchValueStats.to_excel(writer, sheet_name='pitchValueStats')
    dfPlateDisciplineStats.to_excel(writer, sheet_name='plateDisciplineStats')

print('Excel file saved to...', data_dir)

Excel file saved to... C:\Users\jvgas\OneDrive\Desktop\learning\github\baseballStatistics\py4da\baseballStatistics\datasets\jimbo datasets
